<a href="https://colab.research.google.com/github/Nocimaa/Binance_Bot/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv"

--2025-09-17 12:07:07--  https://raw.githubusercontent.com/Nocimaa/Binance_Bot/refs/heads/main/btc_1h_adx_2017-08-17_to_2025-09-17(in).csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28620838 (27M) [text/plain]
Saving to: ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’

btc_1h_adx_2017-08- 100%[===================>]  27.29M   172MB/s    in 0.2s    

2025-09-17 12:07:08 (172 MB/s) - ‘btc_1h_adx_2017-08-17_to_2025-09-17(in).csv’ saved [28620838/28620838]



In [13]:
import pandas as pd
import numpy as np

In [81]:
df = pd.read_csv("btc_1h_adx_2017-08-17_to_2025-09-17(in).csv")

df = df[df["timestamp"] < "2025-09-01 00:00:00+02:00"]

df["label"] = np.log(df["open"].shift(-1) / df["open"])
df = df.drop("timestamp", axis=1)


features = ["open","high","low","close","volume","ATR14","MACD"]
window = 3
for f in features:
    for w in range(1, window+1):
        df[f"{f}_lag{w}"] = df[f].shift(w)


df.dropna(inplace=True)

In [53]:
df.head()

,open,high,low,close,volume,ADX,+DI,-DI,ATR14,BB_width,...,close_lag3,volume_lag1,volume_lag2,volume_lag3,ATR14_lag1,ATR14_lag2,ATR14_lag3,MACD_lag1,MACD_lag2,MACD_lag3
3,4066.00,4088.12,4066.00,4088.10,5.090504,19.814435,15.006875,21.950910,56.892466,0.125607,...,4043.63,3.904708,54.912335,49.206310,59.567272,60.976292,62.194469,-19.635683,-21.766400,-20.330822
4,4063.00,4063.00,4000.01,4023.11,14.970767,20.869256,13.409755,27.587343,59.120862,0.127199,...,4036.30,5.090504,3.904708,54.912335,56.892466,59.567272,60.976292,-16.617763,-19.635683,-21.766400
5,4023.11,4023.11,4000.00,4000.00,6.477680,21.848881,13.018320,26.783322,56.548657,0.127527,...,4074.00,14.970767,5.090504,3.904708,59.120862,56.892466,59.567272,-19.248307,-16.617763,-19.635683
6,4000.00,4023.11,3934.43,3951.49,22.207985,23.613933,11.616981,31.859436,58.843753,0.127666,...,4088.10,6.477680,14.970767,5.090504,56.548657,59.120862,56.892466,-22.933453,-19.248307,-16.617763
7,3914.59,4023.11,3911.79,3989.17,7.695301,25.496193,10.141237,30.395799,62.592057,0.123878,...,4023.11,22.207985,6.477680,14.970767,58.843753,56.548657,59.120862,-29.429070,-22.933453,-19.248307


In [54]:
y = df["label"]
X = df.drop("label", axis=1)

In [55]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

k = 20
selector = SelectKBest(score_func=f_regression, k=k)
X_new = selector.fit_transform(X, y)

scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [74]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled  = y_scaler.transform(y_test.values.reshape(-1,1))



X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
y_test_tensor  = torch.tensor(y_test_scaled, dtype=torch.float32).to(device)

In [75]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return self.fc4(x)

model = RegressionNet(input_dim=k)
model = model.to(device)

In [76]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
batch_size = 256

In [77]:
for epoch in range(epochs):
    permutation = torch.randperm(X_train_tensor.size(0))
    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Évaluation train
    with torch.no_grad():
        preds = model(X_train_tensor)
        mse = criterion(preds, y_train_tensor).item()
    print(f"Epoch {epoch+1}/{epochs}, Train MSE={mse:.4f}")

Epoch 1/50, Train MSE=0.0226
Epoch 2/50, Train MSE=0.0210
Epoch 3/50, Train MSE=0.0167
Epoch 4/50, Train MSE=0.0158
Epoch 5/50, Train MSE=0.0144
Epoch 6/50, Train MSE=0.0157
Epoch 7/50, Train MSE=0.0135
Epoch 8/50, Train MSE=0.0129
Epoch 9/50, Train MSE=0.0144
Epoch 10/50, Train MSE=0.0122
Epoch 11/50, Train MSE=0.0139
Epoch 12/50, Train MSE=0.0128
Epoch 13/50, Train MSE=0.0127
Epoch 14/50, Train MSE=0.0116
Epoch 15/50, Train MSE=0.0119
Epoch 16/50, Train MSE=0.0151
Epoch 17/50, Train MSE=0.0117
Epoch 18/50, Train MSE=0.0121
Epoch 19/50, Train MSE=0.0116
Epoch 20/50, Train MSE=0.0126
Epoch 21/50, Train MSE=0.0104
Epoch 22/50, Train MSE=0.0118
Epoch 23/50, Train MSE=0.0116
Epoch 24/50, Train MSE=0.0112
Epoch 25/50, Train MSE=0.0126
Epoch 26/50, Train MSE=0.0125
Epoch 27/50, Train MSE=0.0174
Epoch 28/50, Train MSE=0.0102
Epoch 29/50, Train MSE=0.0214
Epoch 30/50, Train MSE=0.0115
Epoch 31/50, Train MSE=0.0105
Epoch 32/50, Train MSE=0.0090
Epoch 33/50, Train MSE=0.0094
Epoch 34/50, Train 

In [80]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import torch

with torch.no_grad():
    # prédictions sur GPU
    preds = model(X_test_tensor)

    # ramener sur CPU avant conversion en numpy
    preds_cpu = preds.cpu().numpy()
    y_true_cpu = y_test_tensor.cpu().numpy()

    # si tu as utilisé un scaler pour y
    preds_orig = y_scaler.inverse_transform(preds_cpu)
    y_true_orig = y_scaler.inverse_transform(y_true_cpu)

    # calcul métriques
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    import numpy as np

    mse_test = mean_squared_error(y_true_orig, preds_orig)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(y_true_orig, preds_orig)

print(f"\nTest MSE (original scale) = {mse_test:.4f}")
print(f"Test RMSE (original scale) = {rmse_test:.4f}")
print(f"Test MAE (original scale)  = {mae_test:.4f}")




Test MSE (original scale) = 0.0000
Test RMSE (original scale) = 0.0008
Test MAE (original scale)  = 0.0003


In [64]:
2**16

65536